In [1]:
#=======================================================================================================================================
# Huggingface load_dataset 으로 MLM 훈련 하기
#
# => load_dataset 으로 wiki 말뭉치를 로딩하고, 이를 토크화 시키고, 
# input_ids 에 대해 15% 확률로 [MASK]를 씌워서, 실제 모델을 훈련시키는 예제 
#
# => MLM 훈련 말뭉치는 bongsoo/moco-corpus-kowiki202206 사용, 평가 말뭉치는 bongsoo/bongevalsmall 사용
#
# 출처 : https://wikidocs.net/166817
#=======================================================================================================================================

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, BertTokenizerFast, BertConfig, BertForMaskedLM

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [2]:
# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
#input_corpus = "../../data11/my_corpus/my/pre-kowiki-20220620-1줄.txt"
#input_corpus = "bongsoo/moco-corpus"  # huggingface에 등록된 말뭉치 이용
input_corpus = "../../data11/my_corpus/re-moco-corpus2.txt"

# eval 말뭉치 
#eval_corpus = "../../data11/my_corpus/bong_small_eval.txt"
eval_corpus = "bongsoo/bongevalsmall"

# 기존 사전훈련된 모델
#model_path = "../../data11/model/bert/mbertV2.0"
model_path = "jinmang2/kpfbert"

# 기존 사전 + 추가된 사전 파일
#vocab_path="../tokenizer_sample/moco-vocab/moco-corpus-kowiki202206-42000"
vocab_path="../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add"

# 출력
OUTPATH = '../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add/'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 32       # batch_size
token_max_len = 128   # token_seq_len
epoch = 10            # epoch
lr = 5e-5             # learning rate(기본:5e-5)
############################################################################


device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="distilbert-MLM-Trainer", logfilename="../../log/distilbert-MLM-Trainer")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../../log/distilbert-MLM-Trainer_2023-03-24.log


In [3]:
# tokeinzier 생성
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨
tokenizer = BertTokenizerFast.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
#tokenizer = AutoTokenizer.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
# fast 토크너나이즈인지 확인
print(f'{vocab_path} is_fast:{tokenizer.is_fast}')

# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('*special_token_size: {}, *tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('*vocab_size: {}'.format(vocab_size))
print('*tokenizer_len: {}'.format(len(tokenizer)))

# 모델 로딩 further pre-training 
#config = BertConfig.from_pretrained(model_path)
model = BertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
#model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')    

#################################################################################
# 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
# 재설정하지 않으면, 다음과 같은 에러 발생함
# CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
#  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
#
#     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
#     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
#################################################################################
model.resize_token_embeddings(len(tokenizer))

model.to(device)

../../data11/model/bert/kpfbert/re-moco-corpus2-20000-vocab-add is_fast:True
*special_token_size: 5, *tokenizer.vocab_size: 43889
*vocab_size: 43890
*tokenizer_len: 43889


Some weights of BertForMaskedLM were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(43889, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

In [4]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 훈련 말뭉치 로딩
#train_dataset = load_dataset(input_corpus)
train_dataset = load_dataset("text", data_files=input_corpus) # text 로컬 파일 로딩

# 평가 말뭉치 로딩
eval_dataset = load_dataset(eval_corpus)

# train_dataset 출력해봄
print(f"train_dataset=======================================")
print(train_dataset)
print(train_dataset['train']['text'][0:3])

print(f'\r\n\r\n')

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset['test']['text'][0:3])

[Kss]: Using custom data configuration default-a65b4f99408fe421


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /MOCOMSYS/.cache/huggingface/datasets/text/default-a65b4f99408fe421/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Downloading:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--bongevalsmall-5b85d54295d1a976/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

train_dataset=======================================
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3337234
    })
})
['example of an actor using the system It is not a use case and in most projects it does not survive', '37 서버명 업무내용 제조사 비고 형상관리 형상관리업무 HP 행정정보1 행정정보업무 HP 행정정보2 행정정보업무 HP 행정정보테스트 HP 파생상품1 파생상품업무 HP 파생상품2 파생상품업무 HP 파생상품3 파생상품업무 HP 파생상품테스트 파생상품테스트업무 HP EDDDB1 EDD DB업무 HP RAC EDDDB2 EDD DB업무 HP EDDWAS1 EDD WAS업무 HP EDDWAS2 EDD WAS업무 HP EDDWEB1 EDD WEB업무 HP EDDWEB2 EDD WEB업무 HP EDD테스트 EDD테스트업무 HP IFRS DB업무 HP IFRS테스트 IFRS DB테스트업무 HP', '변경에 따른 관련 법령 예 전자정부법 IT자원의 공동활용신설 공개SW의 이용 및 라이선스 등 하위규정 등']



eval_dataset========================================
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 200
    })
})
['국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야 한다는 자기 안이 있어야 한다 고 말했다 ', '국책연구기관의 한 관계자는  위원회가 전문성과 대표성을 갖추고 본연의 장점을 최대한 살리기 위해서는 외부 감시와 통제가 보다 활성화돼야 한다 고 지적했다 ', '게임업계 관계자는  현장 수요보다 의료진 

In [5]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result


# batched=True 하면 빠른 tokenizer 이용(Rust)
%time train_dataset_fast = train_dataset.map(tokenizer_function, batched=True)

%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

'''
%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)
print(tokenized_dataset_fast['train']['text'][0:2])
'''

  0%|          | 0/3338 [00:00<?, ?ba/s]

CPU times: user 17min 4s, sys: 24min 39s, total: 41min 43s
Wall time: 3min 33s


  0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: user 116 ms, sys: 52.5 ms, total: 169 ms
Wall time: 42.8 ms


"\n%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)\nprint(tokenized_dataset_fast['train']['text'][0:2])\n"

In [6]:
print(f"train_dataset_fast=======================================")
print(train_dataset_fast)
print(f'*fast_len:{len(train_dataset_fast["train"])}, len:{len(train_dataset["train"])}')  # fast_dataset과 dataset 길이를 비교함
print(train_dataset_fast['train'][0:2])

print(f'\r\n\r\n')

print(f"eval_dataset_fast=======================================")
print(eval_dataset_fast)
print(f'*fast_len:{len(eval_dataset_fast["test"])}, len:{len(eval_dataset["test"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['test'][0:2])

train_dataset_fast=======================================
DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3469899
    })
})
*fast_len:3469899, len:3337234
{'text': ['example of an actor using the system It is not a use case and in most projects it does not survive', '37 서버명 업무내용 제조사 비고 형상관리 형상관리업무 HP 행정정보1 행정정보업무 HP 행정정보2 행정정보업무 HP 행정정보테스트 HP 파생상품1 파생상품업무 HP 파생상품2 파생상품업무 HP 파생상품3 파생상품업무 HP 파생상품테스트 파생상품테스트업무 HP EDDDB1 EDD DB업무 HP RAC EDDDB2 EDD DB업무 HP EDDWAS1 EDD WAS업무 HP EDDWAS2 EDD WAS업무 HP EDDWEB1 EDD WEB업무 HP EDDWEB2 EDD WEB업무 HP EDD테스트 EDD테스트업무 HP IFRS DB업무 HP IFRS테스트 IFRS DB테스트업무 HP'], 'input_ids': [[2, 36567, 7749, 13659, 36617, 7005, 36501, 8019, 28902, 19636, 14369, 21957, 569, 36474, 36983, 8393, 8063, 37231, 38259, 19175, 36845, 21957, 39085, 42337, 8855, 3], [2, 8470, 12986, 4719, 7380, 4926, 4587, 17022, 37571, 10813, 10985, 10813, 10985, 4806, 4785, 24154, 7252, 22733, 4827, 7252, 22733, 480

In [7]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_train_sample = data_collator(train_dataset_fast['train']['input_ids'][0:2])
mlm_eval_sample = data_collator(eval_dataset_fast['test']['input_ids'][0:2])

print(f"train_dataset_fast(MLM)=======================================")
print(mlm_train_sample['input_ids'][0])
print(train_dataset_fast['train'][0])

print(f'\r\n\r\n')

print(f"eval_dataset_fast(MLM)=======================================")
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast['test'][0])

train_dataset_fast(MLM)=======================================
tensor([    2,     4,  7749, 13659, 36617,  7005, 36501,  8019, 28902, 19636,
        14369, 21957,   569, 36474, 36983,  8393,  8063, 37231,     4, 19175,
        36845, 21957, 39085, 42337,     4,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
 

In [8]:
# 훈련 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#########################################################################################
# hyper parameter 설정
#########################################################################################
epochs = epoch          # epochs
#lr = 3e-5  # 학습률

total_optim_steps = len(train_dataset_fast["train"]) * epochs // batch_size   # 총 optimize(역전파) 스탭수 = 훈련dataset 계수 * epochs // 배치 크기
eval_steps=int(total_optim_steps * 0.05)           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)
save_steps=int(total_optim_steps * 0.1)            # 저장 스탭수 
save_total_limit=2                                 # 마지막 2개 남기고 삭제 

print(f'*total_optim_steps: {total_optim_steps}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}, *save_steps:{save_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                      # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    overwrite_output_dir=True,         
    num_train_epochs=epochs,                # 에폭
    learning_rate=lr,                      # lr: 기본 5e-5
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    save_steps=save_steps,                  # step 수마다 모델을 저장
    save_total_limit=save_total_limit,      # 마지막 두 모델 빼고 과거 모델은 삭제
    evaluation_strategy="steps",            # 평가 전략 : steps
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
train_dataset_fast_input_ids = train_dataset_fast['train']['input_ids']
eval_dataset_fast_input_ids = eval_dataset_fast['test']['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    train_dataset=train_dataset_fast_input_ids,   # 훈련 데이터셋
    eval_dataset=eval_dataset_fast_input_ids      # 평가 데이터셋
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


*total_optim_steps: 1084343, *eval_steps:54217, *logging_steps:54217, *save_steps:108434
*no_cuda: False


Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
# 훈련 시작
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3469899
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1084350
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss,Validation Loss
54217,2.766100,3.206623
108434,2.175900,3.564524
162651,2.015100,3.369909
216868,1.926900,3.420015
271085,1.853300,3.387788
325302,1.806500,3.697214
379519,1.754100,3.510604
433736,1.716400,3.693922
487953,1.675600,3.614106
542170,1.647100,3.530015


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add/checkpoint-108434
Configuration saved in ../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add/checkpoint-108434/config.json
Model weights saved in ../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add/checkpoint-108434/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
Saving model checkpoint to ../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add/checkpoint-216868
Configuration saved in ../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add/checkpoint-216868/config.json
Model weights saved in ../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add/checkpoint-216868/pytorch_model.bin
***** Running Evaluation *

In [ ]:
# 모델 저장
### 전체모델 저장
TMP_OUT_PATH = '../../data11/model/bert/kpfbert-re-moco-corpus2-20000-vocab-add-last/'
os.makedirs(TMP_OUT_PATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(TMP_OUT_PATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = TMP_OUT_PATH
tokenizer.save_pretrained(VOCAB_PATH)
print(f'==> save_model : {TMP_OUT_PATH}')